In [1]:
from jpnotebooks.other.wql import *
import collections
import types
import datetime
import itertools
import time
%load_ext autoreload
%autoreload 2

In [2]:
MIDNA = 'Mîdna'
SERVER = 'Turalyon'
OWNER = "warbill"
GUILD = "Synthesis"
ZONE = "Uldir"
MYTHIC = 5
api_key = "192b298b50ad53497d9f2b4ce7910af2"

try:
    zraw
except NameError:
    zraw = Api(api_key).zones._raw
    
def connect(api_key=api_key):
    global api
    api = Api(api_key)
    api.zones_ = Zones(api, zraw)
    return api

def pd(o):  # printdir
    for a in dir(o):
        if not a.startswith("__"):
            v = getattr(o,a,"<Error>")
            if not isinstance(v, (types.FunctionType, types.MethodType)):
                print(a, "::", v)

                
# TODO: move these to wql.py once I'm confident they work properly


def _clstrfk(events, l):
    try:
        _clstrfk2(events, l)
    except ValueError:
        for c in l:
            for e in c:
                print(e.timestamp, actor(f, e))
            print()
    return l
    
def _clstrfk2(events, l):
    # debug sanity check
    seen = set()
    evd = dict.fromkeys(events, False)
    for c in l:
        for e in c:
            if evd[e] == True:
                raise ValueError("Duplicate event!")
            evd[e] = True
            
def cluster2(events, before=100, after=100):
    l = set()
    for e in events:
        l.add(tuple(events_near(events, e, before, after)))
    _clstrfk(events, l)
    return sorted(map(list,l), key=lambda c: c[0].timestamp)

def cluster(events, before=100, after=100):
    l = cluster2(events, before, after)
    l = [(c[0].timestamp, c) for c in l]
    return sorted(l, key=lambda t: t[0])
   
def hms2(t):
    t = t / 1000
    h = t // 3600
    m = (t-3600*h) // 60
    s = (t-3600*h-60*m)
    ms = (s - int(s))*1000
    return "%.2d:%.2d.%.3d" %(m,s, ms)
                
connect()
api.verbose = False

Requesting '/zones?api_key=192b298b50ad53497d9f2b4ce7910af2'


In [3]:
def deaths_near_bad(f, *bad, before=100, after=10000, cutoff=8):
    fs_deaths = 'type="death"'
    deaths = [e for e in f.events(fs_deaths, cutoff=cutoff) if e.targetID and f.actor(e.targetID).is_player()]
    fs_bads = 'type="damage" AND (%s)' % " OR ".join('ability.name="%s"'%b for b in bad)
    bads = f.events(fs_bads, cutoff=cutoff)
    fails = set()
    for d in deaths:
        t = tuple(events_near(bads, d, before, after)), d
        fails.add(t)
    return fails

def deaths_near_bad2(f, *bad, before=100, after=10000, cutoff=8):
    fails = deaths_near_bad(f, *bad, before=before, after=after, cutoff=cutoff)
    fails2 = set()
    deaths = set()
    for ev, d in fails:
        fails2.update(ev)
        deaths.add(d)
    key = lambda e: e.timestamp
    return sorted(fails2, key=key), sorted(deaths, key=key)

def hit_by_bad(f, *bad, cutoff=8):
    fs = 'type="damage" AND (%s)' % " OR ".join('ability.name="%s"'%b for b in bad)
    return f.events(fs, cutoff=cutoff)

In [4]:
cache = {}
def get(url):
    global cache
    if url in cache:
        return cache[url]
    rv = api.sess.request("GET", url).content.decode()
    cache[url] = rv
    return rv
def query(s):
    return pyquery.PyQuery(s)("#reports-table > tbody > tr")

In [5]:
def throttle(t):
    def _throttle(f):
        l = time.time()
        n = l + t
        def func(*args, **kw):
            nonlocal l, n
            c = time.time()
            s = n-c
            if s > 0:
                time.sleep(s)
            try:
                rv = f(*args, **kw)
            finally:
                l = c
                n = c + t
            return rv
        return func
    return _throttle

In [12]:
def actor(f, id):
    return f.actor(id).name

In [6]:
GUILD = "Nurfed"
SERVER = "Tichondrius"
reports = api.reports(GUILD, SERVER, "US")
rs = reports.find(title="Uldir")

In [25]:
def hms(s):
    h, s = divmod(s, 3600)
    m, s = divmod(s, 60)
    return "%02d:%02d:%02d"%(h,m,s)

def hms2(ms):
    return hms(ms/1000)

In [26]:
fights = api.fights("BJTwtdGVFqnr12kb")
mythrax = fights.find(name="Mythrax")
for f in mythrax:
    start = f.events('type="encounterstart"')[0]['timestamp']
    baddies = hit_by_bad(f, "Obliteration Blast", "Obliteration Beam")
    for b in baddies:
        pid = b['targetID']
        if pid != -1:
            player = actor(f, pid)
            print("%-20s %-20s %s"%(hms2(b.timestamp - start), player, b['ability']['name']))

00:00:24             Rosael               Obliteration Blast
00:00:24             Namiyo               Obliteration Blast
00:00:24             Midnaught            Obliteration Blast
00:00:25             Jukie                Obliteration Blast
00:01:25             Monkerboii           Obliteration Blast
00:01:26             Vandarkholme         Obliteration Blast
00:01:38             Rosael               Obliteration Blast
00:01:58             Akalic               Obliteration Blast
00:02:23             Bladedguilt          Obliteration Beam
00:02:38             Rialai               Obliteration Beam
00:02:38             Qwezek               Obliteration Beam
00:02:38             Yarmageddon          Obliteration Beam
00:02:38             Monkerboii           Obliteration Beam
00:02:38             Vandarkholme         Obliteration Beam
00:02:39             Rialai               Obliteration Beam
00:02:39             Yarmageddon          Obliteration Beam
00:02:39             Monkerboii 

In [22]:
b

{'ability': {'abilityIcon': 'ability_priest_voidentropy.jpg',
  'guid': 273554,
  'name': 'Obliteration Blast',
  'type': 32},
 'absorbed': 57317,
 'amount': 19759,
 'armor': 4128,
 'attackPower': 10234,
 'classResources': [{'amount': 841, 'max': 1000, 'type': 6}],
 'facing': 12,
 'hitPoints': 98639,
 'hitType': 1,
 'itemLevel': 386,
 'mapID': 1155,
 'maxHitPoints': 198960,
 'mitigated': 1929,
 'pin': '0',
 'resourceActor': 2,
 'source': {'guid': 0,
  'icon': 'NPC-warcraft-0',
  'id': -1,
  'name': 'Environment',
  'type': 'NPC'},
 'sourceIsFriendly': False,
 'spellPower': 873,
 'targetID': 11,
 'targetIsFriendly': True,
 'timestamp': 12660801,
 'type': 'damage',
 'unmitigatedAmount': 79005,
 'x': 26320,
 'y': 24256}

In [115]:
fights = api.fights("BJTwtdGVFqnr12kb").find(name="Mythrax")
f = fights.fights[-1]

start_cast = kill.events('type="begincast" AND ability.name="Enticing Essence" AND source.name="Corruption Corpuscle"')
damage = kill.events('type="damage" and target.name="Corruption Corpuscle"')

adds = {e.sourceInstance: {} for e in start_cast}
ts = {e.sourceInstance: e.timestamp for e in start_cast}

for d in damage:
    p = actor(kill, d.sourceID)
    a = d.targetInstance
    g = adds[a]
    if p not in g:
        g[p] = d.timestamp, d.amount
    else:
        if d.timestamp < g[p][0]:
            amt = g[p][1]
            g[p] = d.timestamp, amt + d.amount

for aid, ahits in sorted(adds.items(), key=lambda t: t[0]):
    first = ts[aid]
    hits = []
    for p, (t, a) in ahits.items():
        hits.append((p, (t-first)/1000, a) )
    hits.sort(key=lambda t: t[1])
    print("Add #%d:"%aid)
    for h in hits:
        print("    %-15s %.2f %d"%(h))
        
    

Add #1:
    Adam            1.52 3107
    Aurelas         1.60 3240
    Shalakittena    3.09 837
    Stevex          3.13 1621
    Conrad          3.27 11781
    spiritMROOSE    3.55 7027
    Naenow          4.12 3609
    Mend            5.13 7014
    Leaxaxo         6.97 7566
    Zestycrouton    8.04 44614
    Treant          12.81 1223
Add #2:
    Thecucker       2.42 6278
    Handidh         3.43 2083
    Piecez          4.12 23681
    Voomkin         4.48 2128
    classicwhen     5.36 13440
    Ub              6.73 87267
    Stevex          8.68 2155
    Ovid            9.10 3227
    Treant          12.22 1363
Add #3:
    Stevex          1.72 1643
    Shalakittena    2.06 976
    Aurelas         2.60 6739
    Adam            2.78 3574
    Naenow          3.52 3647
    spiritMROOSE    4.16 19261
    Zestycrouton    6.68 15383
    Mend            7.58 10389
    Baconmuffin     8.17 4085
    Treant          8.17 1187
    Leaxaxo         9.05 3333
Add #4:
    Piecez          1.48 1836


In [124]:
def event_damage(*abilities):
    tmpl = "ability.name=\"%s\""
    al = [tmpl%a for a in abilities]
    base = " OR ".join(al)
    return "(type=\"damage\" AND (%s))"%base
    
def crushing_fear():
    cf = 'ability.name="Crushing Fear"'
    tmpl = '(type="%s")'
    evt = "damage", "applydebuff", "applydebuffstack", "removedebuff"
    fs = "%s AND (%s)"%(cf, " OR ".join(map(tmpl.__mod__, evt)))
    return fs

In [80]:
pstatus = {player.name: collections.defaultdict(lambda: 0) for player in f.friendlies.players().friendlies}
pdamage = {player.name: 0 for player in f.friendlies.players().friendlies}